In [1]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from scipy.stats import linregress

import sys
sys.path.append('..')

import modules.plotting as pl
import modules.general as gen
import modules.signals as sig
import modules.gait_metrics as gm
import modules.assign_sides as asi
import modules.sliding_window as sw
import modules.linear_algebra as lin
import modules.phase_detection as pde

In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [3]:
file_name = '2014-12-08_P004_Post_004'

In [4]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_best_pos = pd.read_pickle(best_pos_paths[0])

## Split trial into separate passes

In [5]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_best_pos = df_best_pos.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_best_pos.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = gen.map_sort(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = gen.group_by_label(df_best_pos, labels)

In [6]:
df_pass = list(pass_dfs)[0]

In [7]:
frames = df_pass.index.values

In [8]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Detect phases

In [9]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

frames_interest = pde.frames_of_interest(signal_l)


In [19]:
df_phase_l = pde.foot_phases(frames_interest, direction_pass, df_assigned.L_FOOT)
df_phase_r = pde.foot_phases(frames_interest, direction_pass, df_assigned.R_FOOT)

In [273]:
df_phase_lr = df_phase_l.join(df_phase_r, lsuffix='_L', rsuffix='_R')

In [188]:
from toolz import pipe, compose
from functools import partial
from numpy.linalg import norm
from scipy.spatial.distance import cdist

In [132]:
df_stance = df_phase[df_phase.phase == 'stance']

In [262]:
def foo(df_stance, column, func):

    series = df_stance.groupby('number')[column].apply(func)

    result_list = [np.median(cdist(a, b)) for a, b in gen.pairwise(series)]

    return np.array(result_list)

In [263]:
stride_lengths = foo(df_stance, 'position', np.stack)

In [264]:
func = compose(to_column, np.stack)
stride_times = foo(df_stance.reset_index(), 'Frame', func) / 30

In [265]:
stride_lengths

array([90.64644247, 78.42079166, 68.27917134])

In [266]:
stride_times

array([0.95      , 1.16666667, 1.03333333])

In [267]:
stride_velocities = stride_lengths / stride_times

In [268]:
stride_velocities

array([95.41730786, 67.21782143, 66.07661742])